<h2>--- Day 7: Some Assembly Required ---</h2>

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/oddrationale/AdventOfCode2015FSharp/master?urlpath=lab%2Ftree%2FDay07.ipynb)

<p>This year, Santa brought little Bobby Tables a set of wires and <a href="https://en.wikipedia.org/wiki/Bitwise_operation">bitwise logic gates</a>!  Unfortunately, little Bobby is a little under the recommended age range, and he needs help <span title="You had one of these as a kid, right?">assembling the circuit</span>.</p>
<p>Each wire has an identifier (some lowercase letters) and can carry a <a href="https://en.wikipedia.org/wiki/16-bit">16-bit</a> signal (a number from <code>0</code> to <code>65535</code>).  A signal is provided to each wire by a gate, another wire, or some specific value. Each wire can only get a signal from one source, but can provide its signal to multiple destinations.  A gate provides no signal until all of its inputs have a signal.</p>
<p>The included instructions booklet describes how to connect the parts together: <code>x AND y -> z</code> means to connect wires <code>x</code> and <code>y</code> to an AND gate, and then connect its output to wire <code>z</code>.</p>
<p>For example:</p>
<ul>
<li><code>123 -> x</code> means that the signal <code>123</code> is provided to wire <code>x</code>.</li>
<li><code>x AND y -> z</code> means that the <a href="https://en.wikipedia.org/wiki/Bitwise_operation#AND">bitwise AND</a> of wire <code>x</code> and wire <code>y</code> is provided to wire <code>z</code>.</li>
<li><code>p LSHIFT 2 -> q</code> means that the value from wire <code>p</code> is <a href="https://en.wikipedia.org/wiki/Logical_shift">left-shifted</a> by <code>2</code> and then provided to wire <code>q</code>.</li>
<li><code>NOT e -> f</code> means that the <a href="https://en.wikipedia.org/wiki/Bitwise_operation#NOT">bitwise complement</a> of the value from wire <code>e</code> is provided to wire <code>f</code>.</li>
</ul>
<p>Other possible gates include <code>OR</code> (<a href="https://en.wikipedia.org/wiki/Bitwise_operation#OR">bitwise OR</a>) and <code>RSHIFT</code> (<a href="https://en.wikipedia.org/wiki/Logical_shift">right-shift</a>).  If, for some reason, you'd like to <em>emulate</em> the circuit instead, almost all programming languages (for example, <a href="https://en.wikipedia.org/wiki/Bitwise_operations_in_C">C</a>, <a href="https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Operators/Bitwise_Operators">JavaScript</a>, or <a href="https://wiki.python.org/moin/BitwiseOperators">Python</a>) provide operators for these gates.</p>
<p>For example, here is a simple circuit:</p>
<pre><code>123 -> x
456 -> y
x AND y -> d
x OR y -> e
x LSHIFT 2 -> f
y RSHIFT 2 -> g
NOT x -> h
NOT y -> i
</code></pre>
<p>After it is run, these are the signals on the wires:</p>
<pre><code>d: 72
e: 507
f: 492
g: 114
h: 65412
i: 65079
x: 123
y: 456
</code></pre>
<p>In little Bobby's kit's instructions booklet (provided as your puzzle input), what signal is ultimately provided to <em>wire <code>a</code></em>?</p>

In [ ]:
open System.Collections.Generic

In [ ]:
let input = File.ReadAllLines @"input/07.txt"

Parse the input as a dictionary of arrays.

In [ ]:
let parse (line: string) =
    let parts = line.Split(" -> ")
    parts.[1], parts.[0].Split(" ")

In [ ]:
let circuit = new Dictionary<string, string[]>(dict [for line in input -> line |> parse])

Create a recursive function but save the results back to the dictionary as you go so you don't have to recalculate the values.

In [ ]:
let rec evaluate (circuit: IDictionary<string, string[]>) (wire: string): uint16 =
    let signalOrWire (s: string) =
        match UInt16.TryParse s with
        | true, signal -> signal
        | _ -> evaluate circuit s
    
    if circuit.[wire].Length = 1 then
        let signal = signalOrWire circuit.[wire].[0]
        circuit.[wire] <- [| signal |> string |]
        signal
    elif circuit.[wire].Length = 2 then
        let signal = ~~~ (signalOrWire circuit.[wire].[1])
        circuit.[wire] <- [| signal |> string |]
        signal
    elif circuit.[wire].[1] = "AND" then
        let signal = (signalOrWire circuit.[wire].[0]) &&& (signalOrWire circuit.[wire].[2])
        circuit.[wire] <- [| signal |> string |]
        signal
    elif circuit.[wire].[1] = "OR" then
        let signal = (signalOrWire circuit.[wire].[0]) ||| (signalOrWire circuit.[wire].[2])
        circuit.[wire] <- [| signal |> string |]
        signal
    elif circuit.[wire].[1] = "LSHIFT" then
        let signal = (signalOrWire circuit.[wire].[0]) <<< (circuit.[wire].[2] |> int)
        circuit.[wire] <- [| signal |> string |]
        signal
    elif circuit.[wire].[1] = "RSHIFT" then
        let signal = (signalOrWire circuit.[wire].[0]) >>> (circuit.[wire].[2] |> int)
        circuit.[wire] <- [| signal |> string |]
        signal
    else
        0us

In [ ]:
#!time
evaluate circuit "a"

956

Wall time: 146.5008ms

<h2 id="part2">--- Part Two ---</h2>

<p>Now, take the signal you got on wire <code>a</code>, override wire <code>b</code> to that signal, and reset the other wires (including wire <code>a</code>).  What new signal is ultimately provided to wire <code>a</code>?</p>

In [ ]:
let circuit2 = new Dictionary<string, string[]>(dict [for line in input -> line |> parse])

In [ ]:
circuit2.["b"] <- [| evaluate circuit "a" |> string |]

In [ ]:
#!time
evaluate circuit2 "a"

40149

Wall time: 6.8115ms

[Prev](Day06.ipynb) | [Next](Day08.ipynb)